# Análisis Estadístico de Métricas XAI

Este notebook realiza un análisis estadístico riguroso de las métricas de explicabilidad obtenidas con Quantus.

**Objetivos:**
1. **Intervalos de Confianza del 95%**: Proporcionan una estimación del rango probable de los valores reales de las métricas.
2. **Tests de Significación Estadística**: Comparación por pares de métodos XAI usando el test de Wilcoxon (no paramétrico, adecuado para muestras pequeñas y distribuciones no normales).
3. **Análisis de Potencia Estadística**: Discusión sobre las limitaciones del tamaño muestral y su impacto en la capacidad de detectar diferencias significativas.

**Requisitos previos:**
- Ejecutar `quantus_evaluation.py` para los 3 datasets (blood, retina, breast) con `--num_samples 100`
- Los archivos JSON deben estar en `outputs/quantus_metrics_{dataset}.json`

**Nota**: Con 100 muestras por dataset, tenemos un tamaño muestral adecuado para análisis estadísticos básicos, aunque la potencia para detectar diferencias pequeñas puede ser limitada.

In [ ]:
# Configuración inicial e importaciones

import os
import json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import wilcoxon
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

# Definir ruta del proyecto (ajustar según tu sistema)
PROJECT_DIR = Path.cwd().resolve()  # Usa el directorio actual
# Alternativa para sistemas Linux: PROJECT_DIR = Path("/home/TFM_Laura_Monne").resolve()

# Configuración de rutas
OUTPUTS_DIR = PROJECT_DIR / "outputs"
OUTPUTS_DIR.mkdir(exist_ok=True)

# Lista de datasets
datasets = ["blood", "retina", "breast"]

# Métricas evaluadas
metric_names = ["faithfulness", "localization", "complexity", "randomization", "robustness"]
metric_labels = {
    "faithfulness": "Fidelidad",
    "localization": "Localización",
    "complexity": "Complejidad",
    "randomization": "Aleatorización",
    "robustness": "Robustez"
}

# Métodos XAI
methods = ["gradcam", "gradcampp", "integrated_gradients", "saliency"]
method_labels = {
    "gradcam": "Grad-CAM",
    "gradcampp": "Grad-CAM++",
    "integrated_gradients": "Integrated Gradients",
    "saliency": "Saliency Maps"
}

# Colores para visualización
method_colors = {
    "gradcam": "#1b9e77",   # teal
    "gradcampp": "#d95f02", # orange
    "integrated_gradients": "#1f77b4", # blue
    "saliency": "#d62728",  # red
}

print(f"✅ Configuración inicial completada")
print(f"📁 Directorio de outputs: {OUTPUTS_DIR}")

In [ ]:
# Carga de resultados Quantus desde los archivos JSON

results_by_dataset = {}
metadata_rows = []

for dataset in datasets:
    json_path = OUTPUTS_DIR / f"quantus_metrics_{dataset}.json"
    if json_path.exists():
        with open(json_path, "r", encoding="utf-8") as f:
            results_by_dataset[dataset] = json.load(f)
        print(f"✅ Cargado: {dataset} ({json_path.name})")

        meta = results_by_dataset[dataset].get("metadata", {})
        metadata_rows.append({
            "Dataset": dataset.upper(),
            "Num samples": meta.get("num_samples"),
            "Sample strategy": meta.get("sample_strategy"),
            "Seed": meta.get("seed"),
            "Target": meta.get("target"),
            "Métodos": ", ".join(meta.get("methods", [])) if meta else None,
        })
    else:
        print(f"⚠️  No encontrado: {json_path.name}")
        print(f"   Ejecuta `python quantus_evaluation.py --dataset {dataset} --num_samples 100` antes de usar este notebook.")

if not results_by_dataset:
    raise ValueError("❌ No se encontraron resultados Quantus. Genera primero los ficheros JSON con quantus_evaluation.py.")

if metadata_rows:
    print("")
    print("Resumen de metadata:")
    display(pd.DataFrame(metadata_rows))

In [ ]:
# Cargar datos individuales (scores por muestra) desde los JSON

def load_individual_scores(dataset_name: str) -> dict:
    """
    Carga los valores individuales (scores) por muestra para cada método y métrica.
    
    Returns:
        dict: {
            method: {
                metric: [score1, score2, ..., scoreN]  # Lista de valores por muestra
            }
        }
    """
    results = results_by_dataset[dataset_name]
    individual_data = {}
    
    for method in methods:
        if method not in results:
            continue
        individual_data[method] = {}
        for metric in metric_names:
            method_metric = results[method].get(metric, None)
            if method_metric is None:
                individual_data[method][metric] = None
            else:
                scores = method_metric.get("scores", [])
                # Filtrar None (valores inválidos) y convertir a numpy array
                valid_scores = [s for s in scores if s is not None]
                if len(valid_scores) == 0:
                    individual_data[method][metric] = None
                else:
                    individual_data[method][metric] = np.array(valid_scores, dtype=float)
    
    return individual_data

# Cargar datos individuales para todos los datasets
individual_scores_by_dataset = {}
for dataset in datasets:
    individual_scores_by_dataset[dataset] = load_individual_scores(dataset)
    print(f"✅ Datos individuales cargados para {dataset}")

print(f"\n📊 Número de muestras válidas por dataset:")
for dataset in datasets:
    for method in methods:
        if method in individual_scores_by_dataset[dataset]:
            for metric in metric_names:
                scores = individual_scores_by_dataset[dataset][method].get(metric)
                if scores is not None:
                    print(f"  {dataset}/{method}/{metric}: {len(scores)} muestras válidas")
                    break

In [ ]:
# Calcular Intervalos de Confianza del 95% (IC95%)

def calculate_confidence_interval(scores: np.ndarray, confidence: float = 0.95) -> tuple:
    """
    Calcula el intervalo de confianza usando la distribución t de Student.
    
    Args:
        scores: Array de valores
        confidence: Nivel de confianza (default: 0.95)
    
    Returns:
        (mean, lower_bound, upper_bound, sem)
    """
    if scores is None or len(scores) == 0:
        return (None, None, None, None)
    
    n = len(scores)
    mean = np.mean(scores)
    sem = stats.sem(scores)  # Error estándar de la media
    
    # Grados de libertad
    df = n - 1
    
    # Valor crítico t para IC95%
    t_critical = stats.t.ppf((1 + confidence) / 2, df)
    
    # Intervalo de confianza
    margin = t_critical * sem
    lower = mean - margin
    upper = mean + margin
    
    return (mean, lower, upper, sem)

# Calcular IC95% para todos los datasets, métodos y métricas
confidence_intervals = {}

for dataset in datasets:
    confidence_intervals[dataset] = {}
    for method in methods:
        if method not in individual_scores_by_dataset[dataset]:
            continue
        confidence_intervals[dataset][method] = {}
        for metric in metric_names:
            scores = individual_scores_by_dataset[dataset][method].get(metric)
            if scores is not None and len(scores) > 1:
                mean, lower, upper, sem = calculate_confidence_interval(scores)
                confidence_intervals[dataset][method][metric] = {
                    "mean": mean,
                    "lower_95": lower,
                    "upper_95": upper,
                    "sem": sem,
                    "n": len(scores)
                }
            else:
                confidence_intervals[dataset][method][metric] = None

print("✅ Intervalos de confianza calculados")

In [ ]:
# Mostrar tablas de Intervalos de Confianza del 95%

def display_confidence_intervals_table(dataset_name: str, metric_name: str):
    """Muestra una tabla con IC95% para una métrica específica."""
    data_rows = []
    
    for method in methods:
        if method not in confidence_intervals[dataset_name]:
            continue
        ci_data = confidence_intervals[dataset_name][method].get(metric_name)
        if ci_data is not None:
            data_rows.append({
                "Método": method_labels.get(method, method),
                "Media": f"{ci_data['mean']:.4f}",
                "IC95% Inferior": f"{ci_data['lower_95']:.4f}",
                "IC95% Superior": f"{ci_data['upper_95']:.4f}",
                "SEM": f"{ci_data['sem']:.4f}",
                "N": ci_data['n']
            })
    
    if data_rows:
        df_ci = pd.DataFrame(data_rows)
        print(f"\n{'='*80}")
        print(f"Intervalos de Confianza del 95% - {dataset_name.upper()} - {metric_labels.get(metric_name, metric_name)}")
        print(f"{'='*80}")
        display(df_ci)
        return df_ci
    return None

# Mostrar IC95% para todas las métricas y datasets
for dataset in datasets:
    print(f"\n{'#'*80}")
    print(f"# INTERVALOS DE CONFIANZA DEL 95% - {dataset.upper()}")
    print(f"{'#'*80}")
    
    for metric in metric_names:
        display_confidence_intervals_table(dataset, metric)

In [ ]:
# Tests de Significación Estadística: Test de Wilcoxon (comparación por pares)

def perform_wilcoxon_tests(dataset_name: str, metric_name: str, alpha: float = 0.05) -> pd.DataFrame:
    """
    Realiza tests de Wilcoxon (signed-rank test) para comparar métodos XAI por pares.
    
    El test de Wilcoxon es no paramétrico y adecuado para:
    - Muestras pequeñas (n < 30)
    - Distribuciones no normales
    - Datos pareados
    
    Args:
        dataset_name: Nombre del dataset
        metric_name: Nombre de la métrica
        alpha: Nivel de significación (default: 0.05)
    
    Returns:
        DataFrame con resultados de los tests
    """
    # Obtener métodos disponibles
    available_methods = [m for m in methods if m in individual_scores_by_dataset[dataset_name]]
    available_methods = [m for m in available_methods 
                        if individual_scores_by_dataset[dataset_name][m].get(metric_name) is not None]
    
    if len(available_methods) < 2:
        return None
    
    # Realizar comparaciones por pares
    test_results = []
    
    for i, method1 in enumerate(available_methods):
        scores1 = individual_scores_by_dataset[dataset_name][method1].get(metric_name)
        if scores1 is None or len(scores1) == 0:
            continue
            
        for method2 in available_methods[i+1:]:
            scores2 = individual_scores_by_dataset[dataset_name][method2].get(metric_name)
            if scores2 is None or len(scores2) == 0:
                continue
            
            # Asegurar que tienen la misma longitud (tomar el mínimo)
            min_len = min(len(scores1), len(scores2))
            s1 = scores1[:min_len]
            s2 = scores2[:min_len]
            
            # Test de Wilcoxon (signed-rank test)
            try:
                statistic, p_value = wilcoxon(s1, s2, alternative='two-sided')
                
                # Determinar significación
                is_significant = p_value < alpha
                
                # Calcular diferencia de medias
                mean_diff = np.mean(s1) - np.mean(s2)
                
                test_results.append({
                    "Método 1": method_labels.get(method1, method1),
                    "Método 2": method_labels.get(method2, method2),
                    "Media 1": f"{np.mean(s1):.4f}",
                    "Media 2": f"{np.mean(s2):.4f}",
                    "Diferencia": f"{mean_diff:.4f}",
                    "Estadístico W": f"{statistic:.2f}",
                    "p-valor": f"{p_value:.4f}",
                    f"Significativo (α={alpha})": "Sí" if is_significant else "No"
                })
            except Exception as e:
                # Si hay un error (p. ej., todas las diferencias son cero)
                test_results.append({
                    "Método 1": method_labels.get(method1, method1),
                    "Método 2": method_labels.get(method2, method2),
                    "Media 1": f"{np.mean(s1):.4f}",
                    "Media 2": f"{np.mean(s2):.4f}",
                    "Diferencia": f"{np.mean(s1) - np.mean(s2):.4f}",
                    "Estadístico W": "N/A",
                    "p-valor": "N/A",
                    f"Significativo (α={alpha})": "Error"
                })
    
    if test_results:
        df_tests = pd.DataFrame(test_results)
        return df_tests
    return None

# Realizar tests de Wilcoxon para todas las métricas y datasets
wilcoxon_results = {}

for dataset in datasets:
    wilcoxon_results[dataset] = {}
    print(f"\n{'#'*80}")
    print(f"# TESTS DE WILCOXON - {dataset.upper()}")
    print(f"{'#'*80}")
    
    for metric in metric_names:
        df_tests = perform_wilcoxon_tests(dataset, metric)
        if df_tests is not None and len(df_tests) > 0:
            wilcoxon_results[dataset][metric] = df_tests
            print(f"\n{metric_labels.get(metric, metric).upper()}:")
            display(df_tests)
        else:
            print(f"\n{metric_labels.get(metric, metric).upper()}: No hay datos suficientes para realizar tests")
            wilcoxon_results[dataset][metric] = None

In [ ]:
# Guardar resultados estadísticos en archivos CSV

print("=== Guardando resultados estadísticos en outputs/ ===\n")

# Guardar intervalos de confianza
for dataset in datasets:
    ci_rows = []
    for method in methods:
        if method not in confidence_intervals[dataset]:
            continue
        for metric in metric_names:
            ci_data = confidence_intervals[dataset][method].get(metric)
            if ci_data is not None:
                ci_rows.append({
                    "Dataset": dataset,
                    "Método": method,
                    "Métrica": metric,
                    "Media": ci_data['mean'],
                    "IC95_Lower": ci_data['lower_95'],
                    "IC95_Upper": ci_data['upper_95'],
                    "SEM": ci_data['sem'],
                    "N": ci_data['n']
                })
    
    if ci_rows:
        df_ci_all = pd.DataFrame(ci_rows)
        ci_path = OUTPUTS_DIR / f"quantus_confidence_intervals_{dataset}.csv"
        df_ci_all.to_csv(ci_path, index=False)
        print(f"📁 Guardado: {ci_path.name}")

# Guardar tests de Wilcoxon
for dataset in datasets:
    for metric in metric_names:
        if metric in wilcoxon_results[dataset] and wilcoxon_results[dataset][metric] is not None:
            df_wilcoxon = wilcoxon_results[dataset][metric]
            wilcoxon_path = OUTPUTS_DIR / f"quantus_wilcoxon_{dataset}_{metric}.csv"
            df_wilcoxon.to_csv(wilcoxon_path, index=False)
            print(f"📁 Guardado: {wilcoxon_path.name}")

print("\n✅ Resultados estadísticos guardados")

In [ ]:
# Visualización: Intervalos de Confianza del 95% por Método y Métrica

def plot_confidence_intervals(dataset_name: str, metric_name: str, figsize=(10, 6)):
    """Genera un gráfico de barras con intervalos de confianza del 95%."""
    data_rows = []
    
    for method in methods:
        if method not in confidence_intervals[dataset_name]:
            continue
        ci_data = confidence_intervals[dataset_name][method].get(metric_name)
        if ci_data is not None:
            data_rows.append({
                "method": method,
                "mean": ci_data['mean'],
                "lower": ci_data['lower_95'],
                "upper": ci_data['upper_95'],
                "sem": ci_data['sem']
            })
    
    if not data_rows:
        print(f"No hay datos para {dataset_name} - {metric_name}")
        return
    
    df_plot = pd.DataFrame(data_rows)
    df_plot = df_plot.sort_values('mean', ascending=True)
    
    fig, ax = plt.subplots(figsize=figsize)
    
    x_pos = np.arange(len(df_plot))
    means = df_plot['mean'].values
    lowers = df_plot['lower'].values
    uppers = df_plot['upper'].values
    
    # Barras de error
    errors = [means - lowers, uppers - means]
    
    bars = ax.barh(x_pos, means, xerr=errors, capsize=5, 
                   color=[method_colors.get(m, 'gray') for m in df_plot['method'].values],
                   alpha=0.7, edgecolor='black', linewidth=1.2)
    
    ax.set_yticks(x_pos)
    ax.set_yticklabels([method_labels.get(m, m) for m in df_plot['method'].values])
    ax.set_xlabel(f'{metric_labels.get(metric_name, metric_name)} (IC95%)', fontsize=12)
    ax.set_title(f'Intervalos de Confianza del 95% - {dataset_name.upper()} - {metric_labels.get(metric_name, metric_name)}', 
                 fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    ax.axvline(x=0, color='black', linewidth=0.8, linestyle='-')
    
    # Añadir valores en las barras
    for i, (mean, lower, upper) in enumerate(zip(means, lowers, uppers)):
        ax.text(mean, i, f' {mean:.3f}', va='center', fontsize=9, fontweight='bold')
    
    plt.tight_layout()
    
    # Guardar figura
    out_path = OUTPUTS_DIR / f"quantus_ci_{dataset_name}_{metric_name}.png"
    fig.savefig(out_path, dpi=300, facecolor='white', bbox_inches='tight')
    print(f"✅ Figura guardada: {out_path.name}")
    
    plt.show()
    plt.close()

# Generar gráficos de IC95% para métricas clave
print("Generando gráficos de Intervalos de Confianza...\n")

key_metrics = ["faithfulness", "robustness", "localization"]  # Métricas más importantes

for dataset in datasets:
    for metric in key_metrics:
        if metric in metric_names:
            plot_confidence_intervals(dataset, metric)

## Discusión: Limitaciones de Potencia Estadística

### Tamaño Muestral y Potencia

**Tamaño muestral actual**: 100 muestras por dataset

**Consideraciones**:

1. **Potencia Estadística Limitada para Diferencias Pequeñas**:
   - Con n=100, la potencia para detectar diferencias pequeñas (efectos pequeños, d < 0.3) es limitada.
   - Para detectar diferencias pequeñas con potencia del 80% (α=0.05), se necesitarían aproximadamente 200-300 muestras.
   - Las diferencias grandes (efectos grandes, d > 0.8) son detectables con n=100.

2. **Tests No Paramétricos (Wilcoxon)**:
   - El test de Wilcoxon es menos potente que tests paramétricos (t-test) cuando los datos son normales.
   - Sin embargo, es más robusto ante violaciones de normalidad y adecuado para muestras pequeñas.
   - Con n=100, el test de Wilcoxon tiene buena potencia para diferencias medianas-grandes.

3. **Corrección por Múltiples Comparaciones**:
   - Se realizan múltiples tests (4 métodos → 6 comparaciones por métrica).
   - Sin corrección (Bonferroni, FDR), aumenta el riesgo de falsos positivos (Type I error).
   - **Recomendación**: Considerar corrección de Bonferroni para análisis más conservadores:
     - α_ajustado = α / número_de_comparaciones
     - Para 6 comparaciones: α_ajustado = 0.05 / 6 ≈ 0.0083

4. **Intervalos de Confianza del 95%**:
   - Los IC95% proporcionan una estimación del rango probable de los valores reales.
   - Con n=100, los IC95% son razonablemente precisos (SEM ≈ σ/√100).
   - Si los IC95% de dos métodos no se solapan, sugiere una diferencia significativa.

### Recomendaciones para Futuros Estudios

1. **Aumentar tamaño muestral a 200-300 muestras** para mejorar la potencia estadística.
2. **Aplicar corrección de Bonferroni o FDR** para controlar el error de tipo I en múltiples comparaciones.
3. **Realizar análisis de potencia a priori** para determinar el tamaño muestral necesario según el tamaño del efecto esperado.
4. **Considerar análisis bayesiano** como alternativa complementaria para comparaciones de métodos.

### Interpretación de Resultados

- **p-valor < 0.05**: Evidencia estadística de diferencia (sin corrección por múltiples comparaciones).
- **IC95% no solapados**: Sugiere diferencia significativa entre métodos.
- **Diferencia de medias pequeña pero significativa**: Puede no ser clínicamente relevante; considerar tamaño del efecto (Cohen's d).
